In [4]:
import os
import os.path as osp
from glob import glob
from typing import List, Dict, Union

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

# Загрузка данных

In [5]:
os.makedirs('data/', exist_ok=True)

In [ ]:
!unzip /content/ИТЭЛМА_ЛЦТ.zip -d data/

In [6]:
DATASET_DIR = "/content/data/ИТЭЛМА_ЛЦТ"

# Формируем основной df

In [7]:
TARGET_CLASSES = ['hypoxia', 'regular']
SENSOR_TYPES = ["bpm", "uterus"]

In [8]:
def load_dataset(
        dataset_dir: str,
        target_classes: List[str] = TARGET_CLASSES,
        sensor_types: List[str] = SENSOR_TYPES) -> pd.DataFrame:
    """
    Загружает данные с датчиков всех пациентов в один DataFrame в длинном формате.

    Параметры:
        dataset_dir: путь к корневой директории с данными
        target_classes: список целевых классов для загрузки
        sensor_types: список типов датчиков для загрузки

    Функция обходит структуру папок:
      dataset_dir/класс/patient_id/тип_датчика/файлы.csv

    Возвращает DataFrame со столбцами:
      - patient_id: идентификатор пациента (папка)
      - class: класс (hypoxia, regular)
      - sensor_type: тип датчика (bpm, uterus)
      - file_id: имя файла без расширения
      - time_index: временной индекс
      - time_sec: время в секундах
      - value: значения с датчика
    """
    records = []

    for target_class in target_classes:
        class_dir = osp.join(dataset_dir, target_class)
        if not osp.exists(class_dir):
            continue

        for folder_id in os.listdir(class_dir):
            folder_path = osp.join(class_dir, folder_id)
            if not osp.isdir(folder_path):
                continue

            for sensor_type in sensor_types:
                sensor_dir = osp.join(folder_path, sensor_type)
                if not osp.exists(sensor_dir):
                    continue

                csv_files = glob(osp.join(sensor_dir, "*.csv"))
                for file_idx, file in enumerate(csv_files):
                    df = pd.read_csv(file)
                    df = df.reset_index().rename(columns={"index": "time_index"})
                    df["patient_id"] = folder_id
                    df["class"] = target_class
                    df["sensor_type"] = sensor_type
                    df["file_id"] = osp.splitext(osp.basename(file))[0]
                    records.append(df)

    full_df = pd.concat(records, ignore_index=True)
    return full_df


In [9]:
df = load_dataset(DATASET_DIR)

In [10]:
df.head()

,time_index,time_sec,value,patient_id,class,sensor_type,file_id
0,0,0.000000,174.767504,2,hypoxia,bpm,20250908-05800001_1
1,1,0.127029,174.767504,2,hypoxia,bpm,20250908-05800001_1
2,2,0.254058,174.767504,2,hypoxia,bpm,20250908-05800001_1
3,3,0.381087,174.767504,2,hypoxia,bpm,20250908-05800001_1
4,4,0.508116,112.768573,2,hypoxia,bpm,20250908-05800001_1


In [8]:
TO_SAVE = True

if TO_SAVE:
    df.to_pickle('df.pkl')

## Merge с метаданными

In [11]:
meta_hyp = pd.read_csv("/content/hypoxia_preprocessed.csv", index_col=0)
meta_reg = pd.read_csv("/content/regular_preprocessed.csv", index_col=0)

In [19]:
df

,time_index,time_sec,value,patient_id,class,sensor_type,file_id
0,0,0.000000,174.767504,2,hypoxia,bpm,20250908-05800001_1
1,1,0.127029,174.767504,2,hypoxia,bpm,20250908-05800001_1
2,2,0.254058,174.767504,2,hypoxia,bpm,20250908-05800001_1
3,3,0.381087,174.767504,2,hypoxia,bpm,20250908-05800001_1
4,4,0.508116,112.768573,2,hypoxia,bpm,20250908-05800001_1
...,...,...,...,...,...,...,...
11029891,1766,443.076923,3.966190,8,regular,uterus,20250908-04700001_2
11029892,1767,443.203952,3.836151,8,regular,uterus,20250908-04700001_2
11029893,1768,443.330981,3.836151,8,regular,uterus,20250908-04700001_2
11029894,1769,443.458010,3.836151,8,regular,uterus,20250908-04700001_2


In [17]:
meta_hyp["class"] = "hypoxia"
meta_reg["class"] = "regular"
metadata = pd.concat([meta_hyp, meta_reg], ignore_index=True)

In [22]:
metadata = metadata.rename(columns={'folder_id': 'patient_id'})

In [26]:
df.patient_id = df.patient_id.astype(str)
metadata.patient_id = metadata.patient_id.astype(str)

In [28]:
# посмотрел че будет, если мерджить - не понравилось, много строковой инфы просто так хранится
df.merge(metadata, on=["patient_id", "class"], how="left")

,time_index,time_sec,value,patient_id,class,sensor_type,file_id,диагноз,Ph,CO2,Glu,LAC,BE
0,0,0.000000,174.767504,2,hypoxia,bpm,20250908-05800001_1,Беременность 40-41 неделя. \nГоловное предлежа...,NaN,NaN,NaN,NaN,NaN
1,1,0.127029,174.767504,2,hypoxia,bpm,20250908-05800001_1,Беременность 40-41 неделя. \nГоловное предлежа...,NaN,NaN,NaN,NaN,NaN
2,2,0.254058,174.767504,2,hypoxia,bpm,20250908-05800001_1,Беременность 40-41 неделя. \nГоловное предлежа...,NaN,NaN,NaN,NaN,NaN
3,3,0.381087,174.767504,2,hypoxia,bpm,20250908-05800001_1,Беременность 40-41 неделя. \nГоловное предлежа...,NaN,NaN,NaN,NaN,NaN
4,4,0.508116,112.768573,2,hypoxia,bpm,20250908-05800001_1,Беременность 40-41 неделя. \nГоловное предлежа...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11068939,1766,443.076923,3.966190,8,regular,uterus,20250908-04700001_2,I своевременные оперативные роды в 39 лет круп...,–,–,–,–,–
11068940,1767,443.203952,3.836151,8,regular,uterus,20250908-04700001_2,I своевременные оперативные роды в 39 лет круп...,–,–,–,–,–
11068941,1768,443.330981,3.836151,8,regular,uterus,20250908-04700001_2,I своевременные оперативные роды в 39 лет круп...,–,–,–,–,–
11068942,1769,443.458010,3.836151,8,regular,uterus,20250908-04700001_2,I своевременные оперативные роды в 39 лет круп...,–,–,–,–,–
